# PP Score Calculations

In [ ]:
import pandas as pd
import numpy as np
import ppscore as pps

df = pd.DataFrame()
df["x"] = np.random.uniform(-2, 2, 1_000_000)
df["error"] = np.random.uniform(-0.5, 0.5, 1_000_000)
df["y"] = df["x"] * df["x"] + df["error"]

In [ ]:
df

In [ ]:
pps.score(df, "x", "y"), pps.score(df, "y", "x")

In [ ]:
pps.predictors(df, "y")

In [ ]:
pps.matrix(df)

In [ ]:
import seaborn as sns
predictors_df = pps.predictors(df, y="y")
sns.barplot(data=predictors_df, x="x", y="ppscore")

In [ ]:
import seaborn as sns
matrix_df = pps.matrix(df)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

In [ ]:
import pickle
from numpy import linalg as LA
import numpy as np

In [ ]:
df = pickle.load(open(f'../data/mimic3/new_extended_data_unique_embed_CliBERT_2m.pickle', 'rb'))
x = df['inputs']
x1 = df['embeds']
x2 = df['embeds_events']
X = np.hstack([x1, x2])
y = LA.norm(X, axis=-1)

import einops
x = einops.rearrange(x, 'x y z -> (x y) z')[:,:390-300]

df_ = pd.DataFrame(x)
# df_["y"] = y
# df_['x'] = x
# df_["x1"] = LA.norm(x1, axis=-1)
# df_["x2"] = LA.norm(x2, axis=-1)

In [ ]:
matrix_df = pps.matrix(df_)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

In [ ]:
sns.set(rc={'figure.figsize':(32,24)})
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="magma", linewidths=0, annot=False)